<a href="https://colab.research.google.com/github/MPrazzoli/corpsol/blob/main/Script_buono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install QuantLib

     |████████████████████████████████| 20.1MB 1.2MB/s 


In [3]:
import os
from math import *
import numpy as np
import pandas as pd
from QuantLib import *
from datetime import date, datetime

path = '/content/drive/MyDrive/nicoDiscount.xlsx'

***Inserimento degli Input***

In [4]:
try:
  underlying = input('Inserire il sottostante: ')
  if underlying == '':
    underlying = 'EUR3M'
except:
  underlying = 'EUR3M'

try:
  amortization_type = input('Inserire il piano di ammortamento: ')
  if amortization_type == '':
    amortization_type = 'Bullet'
except:
  amortization_type = 'Bullet'

try:
  notional = float(input('Inserire il nozionale: '))
except:
  notional = 100000.0

try:
  amortization_rate = float(input('Inserire il tasso annuo: '))
except:
  amortization_rate = .05

try:
  maturity_label = input('Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni): ')
  if maturity_label == '':
     maturity_label = '5Y'
except:
  maturity_label = '5Y'

try:
  ref_date = Date(input('Inserire la data iniziale del finanziamento in formato dd/mm/yyyy: '),'%d/%m/%Y')
  if ref_date == '':
    ref_date = Date(date.today().day,date.today().month,date.today().year)
except:
  ref_date = Date(date.today().day,date.today().month,date.today().year)

try:
  tenor = int(input('Inserire il periodo che intercorre tra le rate in numero di mesi: '))
except:
  tenor = 3

try:
  business_convention = input('Inserire la convenzione:')
  if business_convention == '' or business_convention == 'ModifiedFollowing':
    business_convention = ModifiedFollowing
except:
  business_convention = ModifiedFollowing

try:
  date_gen_rule = input('Inserire la regola per il calcolo delle date:')
  if date_gen_rule == '' or date_gen_rule == 'Forward':
    date_gen_rule = DateGeneration.Forward
  else: 
    date_gen_rule = DateGeneration.Backward
except:
  date_gen_rule = DateGeneration.Forward

Inserire il sottostante: 
Inserire il piano di ammortamento: 
Inserire il nozionale: 100000000
Inserire il tasso annuo: 
Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni): 3Y
Inserire la data iniziale del finanziamento in formato dd/mm/yyyy: 27/04/2021
Inserire il periodo che intercorre tra le rate in numero di mesi: 3
Inserire la convenzione:
Inserire la regola per il calcolo delle date:


***Schedule Date***

In [5]:
calendar = TARGET()
start_date = calendar.advance(ref_date,2,Days)
if maturity_label[-1] == 'M':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Months)
elif maturity_label[-1] == 'Y':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Years)
Tenor = Period(tenor, Months)
endofmonth = False
terminationDateConvention = business_convention
df = pd.DataFrame({'startdate':list(Schedule(start_date,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth))[:-1],'enddate':list(Schedule(start_date,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth))[1:]})

In [6]:
# Calcolo numero delle rate totali dell'ammortamento
if maturity_label[-1] == 'M':
  rate_totali = int(12/tenor)
elif maturity_label[-1] == 'Y':
  rate_totali = int((12/tenor) * int(maturity_label.rsplit(maturity_label[-1], 1)[0]))

***Ammortamento Francese***

In [7]:
def ammortamento_francese(amortization_rate,tenor,rate_totali,notional):
  
  t = (1 + amortization_rate / (12 / tenor)) ** rate_totali
  rata_fra = round(notional * t * (amortization_rate / (12 / tenor)) / (t - 1),2)
  Residuo_fra = notional
  Estinto_fra = 0
  tot_I_fra = 0
  plan_fra = np.array
  plan_fra = [[0,0,0,0,0, Residuo_fra]]
  rate_fra = range(0, rate_totali)
  for count in rate_fra:
    Int_quota_fra = round(Residuo_fra * (amortization_rate*(tenor/12)),2)
    Cap_quota_fra = (rata_fra - Int_quota_fra)
    tot_I_fra = (tot_I_fra + Int_quota_fra)
    Residuo_fra = (Residuo_fra - Cap_quota_fra)
    Estinto_fra = (Estinto_fra + Cap_quota_fra)
    Tot_fra = (Int_quota_fra + Cap_quota_fra)
    plan.append([Int_quota_fra, Cap_quota_fra, Tot_fra, tot_I_fra, Estinto_fra, Residuo_fra])
  index = ['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
  df_fra = pd.DataFrame(plan_fra)
  df_fra.columns = index
  df1_fra = pd.concat([df,df_fra], axis=1)
  df2_fra = pd.concat([df,df_fra['Debito Residuo']], axis=1)[:-1]

  return (df2_fra)

***Ammortamento Italiano***


In [8]:
def ammortamento_italiano(amortization_rate,tenor,rate_totali, notional):

  Residuo_ita = notional
  tot_I_ita = 0
  Estinto_ita = 0
  Tot_ita = 0
  plan_ita = np.array
  plan_ita = [[0,0,0,0,0,Residuo_ita]]
  rate_ita = range(0,rate_totali)
  for count in rate_ita:
    Int_quota_ita = round(Residuo_ita*amortization_rate/(12/tenor),2)
    Cap_quota_ita = round((notional/rate_totali),2)
    tot_I_ita = (tot_I_ita + Int_quota_ita)
    Residuo_ita = (Residuo_ita - Cap_quota_ita)
    Estinto_ita = (Estinto_ita + Cap_quota_ita)
    Tot_ita = (Int_quota_ita + Cap_quota_ita)
    plan_ita.append([Int_quota_ita,Cap_quota_ita,Tot_ita,tot_I_ita,Estinto_ita,Residuo_ita])
  index = ['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
  df_ita = pd.DataFrame(plan_ita)
  df_ita.columns=index
  df1_ita = pd.concat([df,df_ita],axis=1)
  df2_ita = pd.concat([df,df_ita['Debito Residuo']],axis=1)[:-1]
  
  return (df2_ita)
  

***Ammortamento Bullet***


In [9]:
def ammortamento_bullet(amortization_rate,tenor,rate_totali, notional):

  Residuo_bullet = notional
  tot_I_bullet = 0
  plan_bullet = np.array
  plan_bullet = [[0,0,Residuo_bullet]]
  rate_bullet = range(0,rate_totali)
  for count in rate_bullet:
    Int_quota_bullet = round(Residuo_bullet*amortization_rate/(12/tenor),2)
    tot_I_bullet = (tot_I_bullet+Int_quota_bullet)
    plan_bullet.append([Int_quota_bullet,tot_I_bullet,Residuo_bullet])
  index = ['Quota Interesse','Interessi Totali','Debito Residuo']
  df_bullet = pd.DataFrame(plan_bullet)
  df_bullet.columns = index
  df1_bullet = pd.concat([df,df_bullet],axis=1)
  df2_bullet = pd.concat([df,df_bullet['Debito Residuo']],axis=1)
  df2_bullet = df2_bullet[:-1]
  
  return (df2_bullet)

***Output del Modello***


In [10]:
if (amortization_type == 'Francese'):
  df_output = (ammortamento_francese(amortization_rate,tenor,rate_totali,notional))
elif (amortization_type == 'Italiano'):
  df_output = (ammortamento_italiano(amortization_rate,tenor,rate_totali,notional))
elif (amortization_type == 'Bullet'):
  df_output = (ammortamento_bullet(amortization_rate,tenor,rate_totali,notional))
  
print (df_output)

             startdate             enddate  Debito Residuo
0     April 29th, 2021     July 29th, 2021     100000000.0
1      July 29th, 2021  October 29th, 2021     100000000.0
2   October 29th, 2021  January 31st, 2022     100000000.0
3   January 31st, 2022    April 29th, 2022     100000000.0
4     April 29th, 2022     July 29th, 2022     100000000.0
5      July 29th, 2022  October 31st, 2022     100000000.0
6   October 31st, 2022  January 30th, 2023     100000000.0
7   January 30th, 2023    April 28th, 2023     100000000.0
8     April 28th, 2023     July 31st, 2023     100000000.0
9      July 31st, 2023  October 30th, 2023     100000000.0
10  October 30th, 2023  January 29th, 2024     100000000.0
11  January 29th, 2024    April 29th, 2024     100000000.0


***Calcolo delle Fixing Date***

In [11]:
try:
  rule_fixing = input('Inserire la regola per le fixing date: ')
  if rule_fixing == '':
     rule_fixing = '-2BD'
except:
  rule_fixing = '-2BD'

Inserire la regola per le fixing date: 


In [12]:
column_list = ['fixing date','fixing start','fixing end']
for col in column_list:
  df_output[col] = np.zeros

for i, date in enumerate(df_output['startdate']):
  if rule_fixing == '-2BD':
    date_fix = calendar.advance(date, -2, Days)
  else:
    date_fix = calendar.advance(date, -1, Months)
    date_fix = calendar.endOfMonth(date_fix)
    date_fix = calendar.advance(date_fix, -1, Days)

  df_output['fixing date'].iloc[i] = date_fix
  df_output['fixing start'].iloc[i] = calendar.advance(date_fix, 2, Days)
for t, date in enumerate(df_output['fixing start']):
  df_output['fixing end'].iloc[t] = calendar.advance(date, tenor, Months, ModifiedFollowing)


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [13]:
df_output

,startdate,enddate,Debito Residuo,fixing date,fixing start,fixing end
0,"April 29th, 2021","July 29th, 2021",100000000.0,"April 27th, 2021","April 29th, 2021","July 29th, 2021"
1,"July 29th, 2021","October 29th, 2021",100000000.0,"July 27th, 2021","July 29th, 2021","October 29th, 2021"
2,"October 29th, 2021","January 31st, 2022",100000000.0,"October 27th, 2021","October 29th, 2021","January 31st, 2022"
3,"January 31st, 2022","April 29th, 2022",100000000.0,"January 27th, 2022","January 31st, 2022","April 29th, 2022"
4,"April 29th, 2022","July 29th, 2022",100000000.0,"April 27th, 2022","April 29th, 2022","July 29th, 2022"
5,"July 29th, 2022","October 31st, 2022",100000000.0,"July 27th, 2022","July 29th, 2022","October 31st, 2022"
6,"October 31st, 2022","January 30th, 2023",100000000.0,"October 27th, 2022","October 31st, 2022","January 31st, 2023"
7,"January 30th, 2023","April 28th, 2023",100000000.0,"January 26th, 2023","January 30th, 2023","April 28th, 2023"
8,"April 28th, 2023","July 31st, 2023",100000000.0,"April 26th, 2023","April 28th, 2023","July 28th, 2023"
9,"July 31st, 2023","October 30th, 2023",100000000.0,"July 27th, 2023","July 31st, 2023","October 31st, 2023"


Npv Obiettivo: prova

In [14]:
df_output['differenza_giorni'] = (df_output['enddate'] - df_output['startdate']) / 360
df_output['prod'] = (df_output['Debito Residuo'] * df_output['differenza_giorni']) * 0.0001
BPV = df_output['prod'].sum()

jupi

*Class: Market Data*

In [18]:
class historical_fixed_rate(object):

    def __init__(self, name, reference_date = None, currency = None, underlying = None):
        
        try:
            self.reference_date = datetime.strptime(reference_date, '%d/%m/%Y')
        except:
            self.reference_date = None
        
        self.name = name
        self.currency = currency
        self.underlying = underlying
        self.historical_fixed_rate_df = pd.DataFrame

class zero_curve_rate(object):
# chiedere a Mirko l'assegnazione di interp_rule e day_count
    def __init__(self, name, reference_date = None, currency = None, underlying = None, interp_rule = None, day_count = None):
        
        self.name = name
        
        try:
            self.reference_date = datetime.strptime(reference_date, '%d/%m/%Y')
        except:
            self.reference_date = None
      
        self.currency = currency
        self.underlying = underlying
        self.interpolation_rule = interp_rule
        self.day_count = day_count
        self.zero_curve_rate_df = pd.DataFrame

In [19]:
def market_data(path):

  

  estimation_curve = zero_curve_rate(name = 'ESTIMATION')
  estimation_curve.zero_curve_rate_df = pd.read_excel(path, sheet_name='estimation')

  discount_curve = zero_curve_rate(name = 'DISCOUNT')
  discount_curve.zero_curve_rate_df = pd.read_excel(path, sheet_name='discount')

  market_data_dict = {'ESTIMATION': estimation_curve, 'DISCOUNT': discount_curve}

  return(market_data_dict)

In [ ]:
# converto le date della curva importata in Date quantlib
market_dict = market_data(path)
for i, d in enumerate(market_dict['ESTIMATION'].zero_curve_rate_df['DATE']):
  market_dict['ESTIMATION'].zero_curve_rate_df['DATE'].iloc[i] = Date(d.day,d.month,d.year)
for i, d in enumerate(market_dict['DISCOUNT'].zero_curve_rate_df['DATE']):
  market_dict['DISCOUNT'].zero_curve_rate_df['DATE'].iloc[i] = Date(d.day,d.month,d.year)

In [21]:
# funzione per convertire le Date quantlib in numeri ATTENZIONE RESTITUISCE SEMPRE UNA LISTA!!!
def date_to_ordinal(date_to_convert):
  date_number = []
  if type(date_to_convert) == QuantLib.Date:
    date_convert = datetime(date_to_convert.year(),date_to_convert.month(),date_to_convert.dayOfMonth())
    date_number.append(date_convert.toordinal())
  else:
    for i, d in enumerate(date_to_convert):
      date_convert = datetime(d.year(), d.month(), d.dayOfMonth())
      date_number.append(date_convert.toordinal())
  return date_number


In [22]:
def rate_interpoletor(interpolation_type, x_interpolation, y_interpolation, date_to_interpolate):
    interpolation_object = interpolation_type(x_interpolation, y_interpolation)
    rate_interpolated = interpolation_object(date_to_interpolate, allowExtrapolation=True)
    return rate_interpolated

ESTIMATION RATE FUNCTION

In [23]:
def estimation_rate_function(dataframe_curve, date_fixing_start, date_fixing_end, ref_date): # da passare i fixed rate !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
  date_curve_number = date_to_ordinal( dataframe_curve['DATE'])      
  date_fixing_start_number = date_to_ordinal(date_fixing_start)
  date_fixing_end_number = date_to_ordinal(date_fixing_end)
  ref_date_number = date_to_ordinal(ref_date)
  rate_start = []
  for date in date_fixing_start_number:
    rate = rate_interpoletor(KrugerCubic, list(date_curve_number), list(dataframe_curve['RATE']), date)
    rate_start.append(rate)
  discount_start = []
  for i, rate in enumerate(rate_start):
    discount_start.append(exp(-rate / 100 * ((date_fixing_start_number[i] - ref_date_number[0]) / 365))) # convenzione curva di estimation dell'EURIBOR è act/365
  rate_end = []
  for date in date_fixing_end_number:
    rate_end.append(rate_interpoletor(KrugerCubic, list(date_curve_number), list(dataframe_curve['RATE']), date))
  discount_end = []
  for i, rate in enumerate(rate_end):
    discount_end.append(exp(-rate / 100 * ((date_fixing_end_number[i] - ref_date_number[0]) / 365)))
  estimated_rate = []
  for i, date in enumerate(date_fixing_start_number):
    if date <= ref_date_number[0] : 
      estimated_rate.append(0) # bisogna mettere il rate che fa riferimento al sottostante !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    else:
      estimated_rate.append((discount_start[i] / discount_end[i] - 1) / ((date_fixing_end_number[i] - date) / 360)) # convenzione per i discount dell'EURIBOR è act/360
  return estimated_rate

In [36]:
forw_rate = estimation_rate_function(market_dict['ESTIMATION'].zero_curve_rate_df,df_output['fixing start'], df_output['fixing end'], ref_date)


In [31]:
def discount_rate(dataframe_curve, ref_date, dataframe_schedule):
  date_curve_number = date_to_ordinal(dataframe_curve['DATE'])
  end_date_number = date_to_ordinal(dataframe_schedule['enddate'])
  ref_date_number = date_to_ordinal(ref_date)
  settlement_date_number = date_to_ordinal(dataframe_schedule['startdate'][0])
  rate_list = [] 
  for date in end_date_number:
    rate_list.append(rate_interpoletor(KrugerCubic, list(date_curve_number), list(dataframe_curve['RATE']), date))
  settlement_date_rate = rate_interpoletor(KrugerCubic, list(date_curve_number), list(dataframe_curve['RATE']), settlement_date_number[0])
  discount_list = []
  for i, rate_curve in enumerate(rate_list):
    discount_list.append(exp(-rate_curve/ 100 * (end_date_number[i] - ref_date_number[0])/ 365))
  discount_settlement = exp(-settlement_date_rate/ 100 * ((settlement_date_number[0] - ref_date_number[0]) / 365))
  discount_rates = []
  for discount in discount_list:
    discount_rates.append(discount / discount_settlement)
  return discount_rates

In [32]:
discount_factor = discount_rate(market_dict['DISCOUNT'].zero_curve_rate_df,ref_date,df_output)
discount_factor


[1.0014451595745224,
 1.0029251980057587,
 1.0044412642780323,
 1.0058695914332871,
 1.0073487625675843,
 1.0088700878664547,
 1.01032845718949,
 1.0116547309932844,
 1.012987812139482,
 1.0142224381655065,
 1.015393228355791,
 1.0164902929907125]

In [47]:
df_output['diff_giorni'] = (df_output['enddate'] - df_output['startdate']) / 360
df_output['prod'] = (df_output['Debito Residuo'] * df_output['diff_giorni']) 
df_output['Rate'] = forw_rate
df_output['Flows'] = df_output['prod'] * df_output['Rate']
df_output['disc factor'] = discount_factor
df_output['Discounted Flow'] = df_output['Flows'] * df_output['disc factor']


In [50]:
NPV_leg_float = df_output['Discounted Flow'].sum()
NPV_leg_float

-1435449.6506823702

In [48]:
df_output

,startdate,enddate,Debito Residuo,fixing date,fixing start,fixing end,differenza_giorni,prod,Rate,Flows,disc factor,Discounted Flow,diff_giorni
0,"April 29th, 2021","July 29th, 2021",100000000.0,"April 27th, 2021","April 29th, 2021","July 29th, 2021",0.252778,2.52778e+07,-0.005364,-135590,1.001445,-135786,0.252778
1,"July 29th, 2021","October 29th, 2021",100000000.0,"July 27th, 2021","July 29th, 2021","October 29th, 2021",0.255556,2.55556e+07,-0.005340,-136467,1.002925,-136866,0.255556
2,"October 29th, 2021","January 31st, 2022",100000000.0,"October 27th, 2021","October 29th, 2021","January 31st, 2022",0.261111,2.61111e+07,-0.005330,-139172,1.004441,-139790,0.261111
3,"January 31st, 2022","April 29th, 2022",100000000.0,"January 27th, 2022","January 31st, 2022","April 29th, 2022",0.244444,2.44444e+07,-0.005290,-129311,1.005870,-130070,0.244444
4,"April 29th, 2022","July 29th, 2022",100000000.0,"April 27th, 2022","April 29th, 2022","July 29th, 2022",0.252778,2.52778e+07,-0.005180,-130939,1.007349,-131901,0.252778
5,"July 29th, 2022","October 31st, 2022",100000000.0,"July 27th, 2022","July 29th, 2022","October 31st, 2022",0.261111,2.61111e+07,-0.005061,-132146,1.008870,-133318,0.261111
6,"October 31st, 2022","January 30th, 2023",100000000.0,"October 27th, 2022","October 31st, 2022","January 31st, 2023",0.252778,2.52778e+07,-0.004930,-124629,1.010328,-125916,0.252778
7,"January 30th, 2023","April 28th, 2023",100000000.0,"January 26th, 2023","January 30th, 2023","April 28th, 2023",0.244444,2.44444e+07,-0.004618,-112886,1.011655,-114201,0.244444
8,"April 28th, 2023","July 31st, 2023",100000000.0,"April 26th, 2023","April 28th, 2023","July 28th, 2023",0.261111,2.61111e+07,-0.004280,-111751,1.012988,-113202,0.261111
9,"July 31st, 2023","October 30th, 2023",100000000.0,"July 27th, 2023","July 31st, 2023","October 31st, 2023",0.252778,2.52778e+07,-0.003983,-100693,1.014222,-102125,0.252778


In [ ]:
historical_fixed_rates = historical_fixed_rate(name = "HISTORICAL RATE")
  historical_fixed_rates.historical_fixed_rate_df = pd.read_excel(path, sheet_name='fixings')
'HISTORICAL': historical_fixed_rates

In [ ]:
class IRS:

  def __init__(self):

    self.start_date = None
    self.maturity_date = None
    self.tenor = None
    self.business_convention = None
    self.date_gen_rule = None
    self.schedule = pd.DataFrame
    self.eom = None

    
    self.npv = None

In [ ]:
def pricer(path,
           underlying,
           amortization_type,
           notional,
           amortization_rate,
           maturity_label,
           ref_date,
           tenor,
           business_convention,
           date_gen_rule,
           currency = 'EUR',
           calendar = TARGET(),
           end_of_month = False,
           deduction_formula = '-1M LBD/-1BD',
           day_count = Actual360):

    dictionary = market_data(path)
    irs = IRS()
    irs.start_date = calendar.advance(ref_date,2,Days)
    if maturity_label[-1] == 'M':
      irs.maturity_date = calendar.advance(irs.start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Months)
    elif maturity_label[-1] == 'Y':
      irs.maturity_date = calendar.advance(irs.start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Years)
    irs.business_convention = business_convention
    irs.tenor = tenor
    irs.date_gen_rule = date_gen_rule
    irs.eom = end_of_month

# aggiungere altri attribuiti all'irs(calendario,EOM, deduction formula, day count)


    
    irs.schedule = pd.DataFrame({'startdate':list(Schedule(irs.start_date,irs.maturity_date,irs.tenor,calendar,irs.business_convention,irs.business_convention,irs.date_gen_rule,irs.eom))[:-1],'enddate':list(Schedule(irs.start_date,irs.maturity_date,irs.tenor,calendar,irs.business_convention,irs.business_convention,irs.date_gen_rule,irs.eom))[1:]})

In [ ]:
class IRS:

  def __init__(self, leg1_name = 'Receive', leg2_name = 'Pay', leg1_rate_type = 'Fixed', leg2_rate_type = 'Floating'):

    self.start_date = None
    self.leg1_flows = pd.DataFrame
    self.leg2_flows = pd.DataFrame
    self.leg1_name = leg1_name
    self.leg2_name = leg2_name
    self.leg1_rate_type = leg1_rate_type  
    self.leg2_rate_type = leg2_rate_type
    self.npv = None

In [ ]:
def submit(path,
           underlying,
           amortization_type,
           notional,
           amortization_rate,
           maturity_label,
           ref_date,
           tenor,
           business_convention,
           date_gen_rule,
           currency = 'EUR',
           calendar = TARGET(),
           end_of_month = False,
           deduction_formula = '-1M LBD/-1BD',
           day_count = Actual360):

  irs = IRS()

In [ ]:
submit(path = market_data_path, underlying = underlying, amortization_type = amortization_type, notional = notional, amortization_rate = amortization_rate, maturity_label = maturity_label, ref_date = ref_date, tenor = tenor,
       business_convention = business_convention, date_gen_rule = date_gen_rule)

In [ ]:
leg_fixed=FixedRateLeg(schedule,Actual360(),[notional],[0.05])
for i in leg_fixed:
  print(i.date(),i.amount())